In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from joblib import Parallel, delayed
import time

from skimage.transform import rescale

In [2]:
stack = 'MD594'
show_labels = True

In [3]:
labelingViz_root = '/oasis/projects/nsf/csd395/yuncong/CSHL_annotationsViz'
labelingViz_dir = os.path.join(labelingViz_root, stack)
os.system('rm -r %s' % labelingViz_dir)
create_if_not_exists(labelingViz_dir)

'/oasis/projects/nsf/csd395/yuncong/CSHL_annotationsViz/MD594'

In [4]:
downscale_factor = 8

In [5]:
dm = DataManager(stack=stack)

In [6]:
# parallel

In [6]:
def visualize_annotation(sec):
    
    dm.set_slice(sec)
    
    ret = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
    if ret is not None:
        user, ts, _, annotations = ret
        
    ret_autoAnno = dm.load_proposal_review_result('autoAnnotate', 'latest', 'consolidated')
    if ret_autoAnno is not None:
        user2, ts2, _, annotations_autoAnno = ret_autoAnno        

#     ret_autoAnno = None

    if ret is None and ret_autoAnno is None:
        return
        
    img = imread(os.path.join(os.environ['DATA_DIR'], '%(stack)s_lossless_aligned_cropped_downscaled', 
                  '%(stack)s_%(sec)04d_lossless_aligned_cropped_downscaled.jpg') % {'stack': stack, 'sec': sec})

#     viz = img_as_ubyte(rescale(img, 1./downscale_factor))
    viz = img[::downscale_factor, ::downscale_factor].copy()
    
    label_texts = set([])
    
    if ret is not None and len(annotations) > 0:
            
        for ann in annotations:

            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (0,0,255), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (0,0,255), 5)

            if show_labels:
                lx, ly = np.array(ann['labelPos']) / downscale_factor
                cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                            cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)

            label_texts.add(ann['label'])
            
            
    if ret_autoAnno is not None and len(annotations_autoAnno) > 0:
        
        for ann in annotations_autoAnno:
            
#             vertices_ref = np.array(ann['vertices']) / downscale_factor
            
            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (255,0,0), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (255,0,0), 2)
            
            print ann['label']

            if show_labels:
            
                if ann['label'] not in label_texts:            
                    lx, ly = np.array(ann['labelPos']) / downscale_factor
                    cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                                cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)


    if ret is not None and ret_autoAnno is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_%(user2)s_%(ts2)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts, 'user2':user2, 'ts2':ts2})
    elif ret is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts})            
    else:
        output_fn = os.path.join(labelingViz_dir, 
             '%(stack)s_%(sec)04d_%(user2)s_%(ts2)s_viz.jpg'% \
                         {'stack':stack, 'sec':sec, 'user2': user2, 'ts2': ts2})
            
    cv2.imwrite(output_fn, viz[..., ::-1])

In [7]:
first_bs_sec, last_bs_sec = section_range_lookup[stack]

import time
t = time.time()

_ = Parallel(n_jobs=16)(delayed(visualize_annotation)(sec) for sec in range(first_bs_sec, last_bs_sec+1))

# simultaneous write seems fine
sys.stderr.write('generate visualization: %.2f seconds\n' % (time.time() - t)) # ~ 60s/150 sections; ~200s/stack

7n
5N
7n
7n
7n
VLL
7n
LC
LC
VLL
7n
sp5
LC
Tz
Tz
Tz
VLL
sp5
sp5
Amb
5N
7n
5N
7n
7n
7n
Amb
VLL7n
7n
7n
sp5
sp5
VLL
7N
sp5
sp5
sp5
sp5
sp5
sp5
sp5
sp5

sp5
sp5
sp5
RtTg
7n5NPn
5N
VLL
5N
VLL
LRt
LRt
5N
LRt
sp5
R
R
RtTg
Pn


7nLRt
7N
LRt
7N
7N
7N
7N
7N
TzLRt
RtTg
Pn
TzVLL
sp5

7N
7n7N
7nPn
Pn
VLLPn

7N
7N
Tz
sp5
VLL
sp5
VLL
VLL
LCRtTg

sp5sp5
Pn
Pn

7n
7n5NLRt

5N

5N

LC7n

R
RtTg
Tz7n
sp5


7N
7n
sp5
7n
VLL
7n

sp5
5N
RtTg
LC
sp5
R
VLL
sp5
Pn
Amb
VLL
sp5
7N
sp5
7n
5N
7N
Pn

7n
R
RtTg
sp5
VLL
7nsp5
7N
5N
7nR
sp5
LRt
Pn
Tz7n
sp5
RtTg
Pn
7n7N

5N
7nLRt

LRt
LRt
7N
7n
sp5
R
Pn
Tz
7nsp5
LRt

7N
5N
7N
7N
VLL
sp5
R
RtTg
Tz
7N

LRt
7N
5N
Pn
VLL
Pn
Pn

sp5
R
RtTg
7N

7n
5N
5N
7N
Pn
sp5
7n7N
RtTg
RtTg
7n
LRt
RtTg
7N
Pn
7n
sp5
VLL
VLL
Pn
VLLVLL

VLLLCLCAmb
7N
Pn
Pn
Tzsp5
R


generate visualization: 386.20 seconds
username yuncong does not have any annotations for current section 93 
username yuncong does not have any annotations for current section 94 
username yuncong does not have any annotations for current section 97 
username autoAnnotate does not have any annotations for current section 96 
username autoAnnotate does not have any annotations for current section 98 
username yuncong does not have any annotations for current section 95 
username yuncong does not have any annotations for current section 101 
username yuncong does not have any annotations for current section 99 
username yuncong does not have any annotations for current section 103 
username yuncong does not have any annotations for current section 107 
username autoAnnotate does not have any annotations for current section 102 
username yuncong does not have any annotations for current section 105 
username autoAnnotate does not have any annotations for current section 106 
username aut

Tzsp5LC
7N
Amb



sp5
Pn
TzLRt

RtTg
RtTg



7n
7nsp5
7n
7n
7n
5N
7n
LC7n
Pn
Pn
sp5
12N
7n
Amb

5N
5N
sp5
sp5
LRt

sp5

sp5
TzTzR
AP
sp5
sp5
5N
LRt
LRt
R
LRt
7N
sp5
R
7n
RtTg


Pn
RtTg
LRt
5N
sp5
7N
7N
LRt
7N
VLLLRt
Pn
sp5
Pn
7n
sp5
RtTg
Pn
7N
LRt
VLL
7nAP7N
Pn

7N
RtTg
R
Tzsp5
R
TzPnPn
7N
7N


Pn
RtTg
7n
Pn
TzRtTg

R
RtTg


sp5Pn
APAmb
RtTg
RtTg
LCAmb
Tz
7N
7n
RtTg
Pn
sp5
RtTg

7n
sp5
12N
Tz
sp5

sp5
Pn
sp5
Pn
7nR
12N
6N

RtTg
5N
sp5

7n
5N
sp5
R
LRt
R
Pn
Pn
sp5
12N
Amb
12N
LRt
APsp5
sp5
LRt
12N
12N
TzRtTg

Pn
RtTg
APPn
sp5
sp5
7N

12N
LRt
7N
R
Pn

Pn
7nTz
Tz
RtTg
5N
APPn
RtTg
R
7N
sp5RtTg
RtTg
LC
7n
RtTg

Pn
sp5LRt

AP12N
RtTg
Pn

Pn
Pn7n

Pn
sp5
sp5
RtTg

7N
RtTg

sp5
Pn
RtTg
12N
Tz
sp5
Pn
Tz
7nR
12N
6N
Pn
12N
Pn
APTzPnAP

RtTg
R
Tz
RtTg



In [ ]:
########## serial version ##########

first_bs_sec, last_bs_sec = section_range_lookup[stack]

for sec in range(first_bs_sec, last_bs_sec+1):
# for sec in range(181, 262+1):
# for sec in [135]:

    dm.set_slice(sec)
    
    ret = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
    if ret is not None:
        user, ts, _, annotations = ret
        
#     ret_autoAnno = dm.load_proposal_review_result('autoAnnotate', 'latest', 'consolidated')
#     if ret_autoAnno is not None:
#         user2, ts2, _, annotations_autoAnno = ret_autoAnno        
    ret_autoAnno = None

    if ret is None and ret_autoAnno is None:
        continue
    
    print '\n'
    print sec
    
    img = imread(os.path.join(os.environ['DATA_DIR'], '%(stack)s_lossless_aligned_cropped_downscaled', 
                  '%(stack)s_%(sec)04d_lossless_aligned_cropped_downscaled.jpg') % {'stack': stack, 'sec': sec})

#     viz = img_as_ubyte(rescale(img, 1./downscale_factor))
    viz = img[::downscale_factor, ::downscale_factor].copy()
    
    label_texts = set([])
    
    if ret is not None and len(annotations) > 0:
            
        for ann in annotations:

            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (0,0,255), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (0,0,255), 5)

            if show_labels:
                lx, ly = np.array(ann['labelPos']) / downscale_factor
                cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                            cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)

            label_texts.add(ann['label'])
            
            
    if ret_autoAnno is not None and len(annotations_autoAnno) > 0:
        
        for ann in annotations_autoAnno:
            
            
#             vertices_ref = np.array(ann['vertices']) / downscale_factor
            
            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (255,0,0), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (255,0,0), 2)
            
            print ann['label']

            if show_labels:
            
                if ann['label'] not in label_texts:            
                    lx, ly = np.array(ann['labelPos']) / downscale_factor
                    cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                                cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)


    if ret is not None and ret_autoAnno is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_%(user2)s_%(ts2)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts, 'user2':user2, 'ts2':ts2})
    elif ret is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts})            
    else:
        output_fn = os.path.join(labelingViz_dir, 
             '%(stack)s_%(sec)04d_%(user2)s_%(ts2)s_viz.jpg'% \
                         {'stack':stack, 'sec':sec, 'user2': user2, 'ts2': ts2})
        
    cv2.imwrite(output_fn, viz[..., ::-1])

In [ ]:
# For MICCAI paper (image, scoremap, auto annotation contours)

In [32]:
first_bs_sec, last_bs_sec = section_range_lookup[stack]

# for sec in range(first_bs_sec, last_bs_sec+1):
for sec in [141]:

    dm.set_slice(sec)
        
    ret_autoAnno = dm.load_proposal_review_result('autoAnnotate', 'latest', 'consolidated')
    if ret_autoAnno is not None:
        user2, ts2, _, annotations_autoAnno = ret_autoAnno
        
    if ret is None and ret_autoAnno is None:
        continue
    
    print sec
    
    
    dm._load_image(versions=['rgb-jpg'])
    cropped_img = dm.image_rgb_jpg[::8, ::8]
    
    scoremaps_rootdir = '/home/yuncong/csd395/CSHL_scoremaps_lossless/'
    scoremaps_dir = os.path.join(scoremaps_rootdir, stack, '%04d'%sec)

    l = '7N'
    
    scoremap_whole = bp.unpack_ndarray_file(os.path.join(scoremaps_dir, 
                                           '%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s.bp' % \
                                           {'stack': stack, 'sec': sec, 'label': l}))
    # except:
    #     sys.stderr.write('No scoremap of %s exists\n' % (l))
    #     continue


    dataset = stack + '_' + '%04d'%sec + '_roi1'

    interpolation_xmin, interpolation_xmax, \
    interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_dir, 
                                                                     '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                    {'dataset': dataset, 'label': l})).astype(np.int)

    dense_scoremap_lossless = np.zeros((dm.image_height, dm.image_width), np.float32)
    dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
                            interpolation_xmin:interpolation_xmax+1] = scoremap_whole

    scoremap = dense_scoremap_lossless[::8, ::8]
    scoremap_height, scoremap_width = scoremap.shape[:2]

    scoremap_layer_viz = img_as_ubyte(plt.cm.hot(scoremap)[..., :3])
    scoremap_viz = img_as_ubyte(alpha_blending(scoremap_layer_viz,
                                               cropped_img,
                                               .3, 1)[..., :3])
    

#     viz = img_as_ubyte(rescale(scoremap_viz, 1./downscale_factor))
#     viz = scoremap_viz[::downscale_factor, ::downscale_factor]
                
    if ret_autoAnno is not None and len(annotations_autoAnno) > 0:
        
        for ann in annotations_autoAnno:
            
            if ann['label'] == l:
            
                vertices = np.array(ann['vertices']) / downscale_factor
                cv2.polylines(scoremap_viz, [vertices.astype(np.int)], True, (255,0,0), 2)

    #             print ann['label']


141


In [33]:
display(scoremap_viz)

/oasis/projects/nsf/csd395/yuncong/Brain/learning/tmp.jpg